In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv
hv.extension("bokeh")

## Declaring data

For presentation purposes, the dummy data contains time series data with frequency patterns which nicely shows the benefit of radial heatmaps to detect periodicities.

Generate hourly values for 31 days with 7 days a week. A segement represents an hour whereas an annular represents an entire day. 

In [ ]:
days = 31
hours = 24

size = days*hours

def generate_hourly_periodic_data(x):
    periodic_weekly = np.sin(x*2*np.pi / (24*7))
    periodic_daily = np.sin(x*2*np.pi / 24)
    noise = np.random.random(size=x.size)
    
    return periodic_weekly + periodic_daily + noise

x = np.linspace(0, size, size)
y = generate_hourly_periodic_data(x)

date_index = pd.date_range(start="2017-10-01", freq="h", periods=size)
kdim_segment = date_index.strftime("%H:%M")
kdim_annular = date_index.strftime("%A %d")

df = pd.DataFrame({"values": y, "hour": kdim_segment, "day": kdim_annular}, index=date_index)
df.head()

In [ ]:
%%opts RadialHeatMap [width=900 height=900 tools=["hover"] show_nth_label=4 padding_outer=0.05 padding_inner=0.1 separate_nth_segment=4]

import holoviews as hv
hv.extension("bokeh")

hv.RadialHeatMap(df, kdims=["hour", "day"])